In [ ]:
# Dag simplehttpoperator
from airflow import DAG
from airflow.operators.http_operator import SimpleHttpOperator
from datetime import datetime

dag = DAG(
    dag_id='dag',
    schedule_interval='@once',
    start_date=datetime(2022, 11, 1),
)

t1 = SimpleHttpOperator(
    task_id='get_num',
    method='GET',
    http_conn_id='http_default',
    endpoint='integers/?num=1&min=1&max=5&col=1&base=2&format=plain',
    xcom_push=True,
    dag=dag)

In [ ]:
# randint to xcom
from airflow import DAG
from datetime import timedelta
from airflow.utils.dates import days_ago
from airflow.models import BaseOperator
from random import randint

class DummyOperator(BaseOperator):

    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)

    def execute(self, context):
        return randint(0,9)

dag = DAG('dag',schedule_interval='@daily', start_date=days_ago(1))
t1 = DummyOperator(task_id='task_1', dag=dag)
t2 = DummyOperator(task_id='task_2',dag=dag)

t1 >> t2

In [ ]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.models import Variable
from airflow.models import Connection
import json
from datetime import datetime

# create conection
c = Connection(
  conn_id="custom_conn_id",
  host="google.com",
  login="user",
  password="12345"
)

def save_into_variables(**context):
  con_str = f'"conn_id":{c.conn_id}, "host":{c.host}, "login":{c.login}, "password":{c.password}'
  Variable.set('my_conn', json.dumps('{'+con_str+'}', ))


dag = DAG('dag', 
          start_date=datetime(2021, 1, 1),
          schedule_interval='@once')

set_varible = PythonOperator(
    task_id='set_data',
    python_callable=save_into_variables,
    dag=dag
)

In [ ]:
import airflow
from airflow import DAG
from airflow.sensors.http_sensor import HttpSensor


dag = DAG('dag',schedule_interval='@daily', start_date=airflow.utils.dates.days_ago(1))

def response_check(response, task_instance):
  if int(response.text) == 5:
    return True
  return False

sensor = HttpSensor(
    task_id='http_sensor',
    http_conn_id='http_default',
    endpoint='integers/?num=1&min=1&max=5&col=1&base=10&format=plain',
    response_check=response_check,
    poke_interval=10,
    timeout=60,
    dag=dag)

In [ ]:
from airflow import DAG
from datetime import datetime
from airflow.operators.dummy_operator import DummyOperator

def create_dag(dag_id,
               default_args,
               schedule='@daily'):

    dag = DAG(dag_id,
              schedule_interval=schedule,
              default_args=default_args)

    with dag:
      task_list=list()
      for i in range(0, 10):
        task_list.append(DummyOperator(task_id=f'task_{i}', dag=dag))

    return dag


# generate DAGS via range()
for n in range(0, 5):
    dag_id = f'dag_{n}'

    # defaults
    default_args = {'owner': 'airflow',
                    'start_date': datetime(2021, 1, 1)
                    }
    
    globals()[dag_id] = create_dag(dag_id, default_args)
